# Modeling Internal Representation

A useful abstraction is thinking of each of the opponent's keywords as a random variable. We know the opponent has a keyword card with a certain number of fixed keywords, but not what the keywords are. However, we know they must be *a* word. So we may as well think of each of their words as having some probability of being each word possible, and refine the probability values based on context and revealed information.

Somewhat less obvious is that we may do the same for our own keywords; for each word, the distribution would simply have probability 1 for the keyword, and 0 for every other word.

# Guessing with Backtracking

To make a good guess, the objective would be to maximize the probability of our guess; this gives the highest likelihood of being correct and raises our expected score through interceptions.

Say we have keywords ["GARDEN", "MUSKET", "BOWL", "BAT"] and clues ("wing", "leaf", "powder"). We might notice right away that "wing" makes a lot of sense with "BAT", so we might choose the first code number to be index 3 and remove both from consideration. This simplifies our consideration to ["GARDEN", "MUSKET", "BOWL"] and clues ("leaf", "powder"), where we might identify more easily now that "leaf" corresponds to "GARDEN". Again, we might choose 0 as our next code number and remove both from consideration. This yields ["MUSKET", "BOWL"] and clues ("powder"). It is much easier to identify that "powder" probably corresponds to "MUSKET". We notice that guessing has somewhat of a recursive property. 

Let's think about how this might be represented mathematically. We might say that each keyword has a random variable associated with it

> K_0, K_1, K_2, K_3

We might then say the each code number has a random variable associated with it
> C_0, C_1, C_2

and each corresponding clue random variables (M is for "message").

> M_0, M_1, M_2 

Then we wish to maximize

> P(M_0="wing", M_1="leaf", M_2="powder").

To do so, we take the maximum of

> P(C_0=i) * P(M_0="wing", M_1="leaf", M_2="powder" | C_0=i) from i = 0 to 3.

This yields

> P(C_0=i) * P(M_0="wing", M_1="leaf", M_2="powder", C_0=i) / P(C_0=i)

The P(C_0=i) terms cancel! To continue, we find

> P(M_0="wing", M_1="leaf", M_2="powder", C_0=i)

> =

> P(M_1="leaf", M_2="powder" | M_0="wing", C_0=i) * P(M_0="wing",  C_0=i)

This is where we recall the example; when we thought about the remaining words, we didn't consider the fact that we had assigned one of the previous clues a code number. That was the utility of the reduction, that it made it more simple to think about; it follows then that the probability of the remaining M_1 and M_2 is independent of the choice of M_0 and C_0, and we may remove that condition.

> P(M_1="leaf", M_2="powder") * P(M_0="wing",  C_0=i) 

We see a recurrence! In its entirety we write it as 

> f(P(M_0, M_1, M_2)) = max(f(P(M_1, M_2=)) * P(M_0,  C_0=i)) from i = 0 to 3

We may calculate

> P(M_0,  C_0=i)

with our random variable representations! 

We might be able to optimize with Dynamic Programming but let's worry about that later if it proves necessary.

In [30]:
import math

class RandomVariableTracker:
    def __init__(self, random_vars: dict):
        self.random_vars = random_vars
        
    def clue_probability(self, clue: str, var_index: int):
        # this would need to be more refined for the general case
        return self.random_vars[var_index][clue] # P(X_i = clue)
        
    def max_log_probability_guess(self, clues: tuple[str], var_indices=None):
        if not clues:
            return 0.0, tuple()
        
        var_indices = var_indices if var_indices is not None else tuple(range(len(self.random_vars))) 
        clue, *remaining_clues = clues
        remaining_clues = tuple(remaining_clues)
                
        max_log_probability = -math.inf
        best_guess = None
        for i, var_index in enumerate(var_indices):
            remaining_var_indices = var_indices[:i] + var_indices[i + 1:]
            
            clue_probability = self.clue_probability(clue, var_index)
            max_subproblem_log_probability, guess = self.max_log_probability_guess(remaining_clues, remaining_var_indices)
            log_probability = max_subproblem_log_probability + (math.log(clue_probability) if clue_probability != 0 else -math.inf)
            
            if log_probability > max_log_probability:
                max_log_probability = log_probability
                best_guess = (var_index,) + guess
                
        return max_log_probability, best_guess

In [31]:
import decryptogame as dg
from collections import defaultdict
from itertools import permutations

# let's see if our recurrence yields a good guessing heuristic!

# generate random keywords
keywords = next(dg.generators.RandomKeywordCards())[0]
print(keywords)

# initialize tracker
random_vars = [defaultdict(float) for _ in range(len(keywords))]

for random_var, keyword in zip(random_vars, keywords):
    random_var[keyword] = 1.0

tracker = RandomVariableTracker(random_vars)

# try every code
codes = list(permutations(range(len(keywords)), 3))
clues = [tuple(keywords[code_num] for code_num in code) for code in codes]
print(clues[:5])

log_probabilities_and_guesses = [tracker.max_log_probability_guess(clue) for clue in clues]

probabilities = [math.exp(log_probability) for log_probability, _ in log_probabilities_and_guesses]
guesses = [guess for _, guess in log_probabilities_and_guesses]

# check that probabilities made sense and that each guess was correct
print(set(probabilities)) # should all be 1
print(guesses == codes) # should be True

('BUTTON', 'GASOLINE', 'THEATRE', 'AQUARIUM')
[('BUTTON', 'GASOLINE', 'THEATRE'), ('BUTTON', 'GASOLINE', 'AQUARIUM'), ('BUTTON', 'THEATRE', 'GASOLINE'), ('BUTTON', 'THEATRE', 'AQUARIUM'), ('BUTTON', 'AQUARIUM', 'GASOLINE')]
{1.0}
True
